<div style="text-align: center;">
    <h1>Correction localisation des entreprises</h1>
</div>


Certains établissements sont mal localisés (au point (0,0)).

Pourquoi ? Quelques hypothèses :

- Problème de manipulation de données 
- Leur ville n'est pas référencée 
- autre ...

On va tenter de corriger cela.

In [41]:
import pandas as pd 
import geopandas as gpd
import folium
import numpy as np
import shapely
from shapely import Point,Polygon
import random

# 1. Quels sont  les établissements concernés ?

Et est ce que le problème concerne beaucoup d'établissements ?

## 1.1 Etablissements avec un doublon

In [2]:
# lecture du fichier
df = pd.read_parquet("../Données nationales/RNE_Sirene.parquet")

In [3]:
siret_example = 98166338800016

lat , lon = df[df.siret == siret_example].y_latitude.values[0],df[df.siret == siret_example].x_longitude.values[0]

m = folium.Map(location=(lat,lon))

marker = folium.Marker(location=(lat,lon))

marker.add_to(m)

m

C'est étonnant car dans les cartes éditées précédemment, cet établissement était localisé au point (0,0).
Ici il est au point correct.

In [4]:
df[df.siret == siret_example]

,siret,qualite_xy,y_latitude,x_longitude,nomCommercial,adresse,codeInseeCommune,codeApe,diffusionCommerciale,confiance
1496031,98166338800016,0.0,43.624496,3.860749,n.s. cf. Pappers,1742 AV DE TOULOUSE 34070 MONTPELLIER,34172,47.11D,True,0
17630212,98166338800016,0.0,0.000000,0.000000,n.s. cf. Pappers,1742 AV DE TOULOUSE 34070 MONTPELLIER,34172,47.11D,True,0


On a l'explication : pour un même établissement il y a deux localisations.
Après vérification, dans la carte éditée précédemment il y a bien deux marqueurs pour un même siret.

Est ce que ce cas de figure concerne tous les établissements mal localisés ?
(*dit autrement : est ce que tous les établissements localisés en (0,0) sont dupliqués*)

On peut pour commencer supprimer toutes les lignes d'entreprises qui sont en doublon.

In [5]:
df = df.sort_values(by = ["siret","y_latitude"],ascending=False)

start = df.shape[0]

# on supprime la deuxième occurence de chaque siret (=la plus petite, =(0,0))
df = df.drop_duplicates(subset="siret",keep="first")
end = df.shape[0]

print("On a enlevé {:,} établissements, soit une réduction de {:.2%}.".format(start-end,(start-end)/start))

On a enlevé 1,587,395 établissements, soit une réduction de 8.95%.


Verif

In [6]:
df[df.siret == siret_example]

,siret,qualite_xy,y_latitude,x_longitude,nomCommercial,adresse,codeInseeCommune,codeApe,diffusionCommerciale,confiance
1496031,98166338800016,0.0,43.624496,3.860749,n.s. cf. Pappers,1742 AV DE TOULOUSE 34070 MONTPELLIER,34172,47.11D,True,0


## 1.2 Etablissements autres

In [7]:
cp = df[(df.y_latitude == 0) & (df.x_longitude == 0)]

nb , nb_unique = cp.shape[0], cp.siret.unique().shape[0]

print("Il y a {:,} entreprises mal localisées dont {:,} entreprises uniques ({:.2%}).".format(nb,nb_unique,nb_unique/nb))

print("Cela représente {:.2%} des entreprises uniques.".format(nb_unique/df.siret.unique().shape[0]))

del cp

Il y a 1,369,686 entreprises mal localisées dont 1,369,686 entreprises uniques (100.00%).
Cela représente 8.49% des entreprises uniques.


In [9]:
df[df.siret == 98298254800017]

,siret,qualite_xy,y_latitude,x_longitude,nomCommercial,adresse,codeInseeCommune,codeApe,diffusionCommerciale,confiance
17728937,98298254800017,0.0,0.0,0.0,,MARSEILLE 9,13209,68.20B,True,0


Pourquoi ce siret est mal localisé ?

Comment est ce qu'il est dans la bdd d'origine ?

In [10]:
# paramètres
url = "https://www.data.gouv.fr/fr/datasets/r/ba6a4e4c-aac6-4764-bbd2-f80ae345afc5"
id_doc = url.split('/')[-1]
usecols = ["siret","qualite_xy","x_longitude","y_latitude"]
dtypes = {"siret":np.int64,"qualite_xy":np.int8,"x_longitude":np.float64,"y_latitude":np.float64}


df_sirene = pd.read_csv(url,compression='zip',sep=';',usecols=usecols,dtype=dtypes)

df_sirene[df_sirene.siret == 98298254800017 ]

,siret,qualite_xy,y_latitude,x_longitude


IL n'est pas référencé dans la bdd d'origine, pourtant il existe bien (cf. [Pappers](https://www.pappers.fr/entreprise/galise-dorothee-982982548)).

Pourquoi est ce qu'on ne lui pas attribué un point aléatoire dans sa commune ?

In [17]:

df_loc = gpd.read_parquet("../Données nationales/populationLocalisationCommunes.parquet")

codeInseeCommune_ex = df[df.siret ==  98298254800017].codeInseeCommune.values[0]

df_loc[df_loc.codeCommune == codeInseeCommune_ex]


,nomCommune,codeCommune,populationCommune,geometryCommune
4415,Marseille 9e Arrondissement,13209,76053,"MULTIPOLYGON (((5.45224 43.25746, 5.44966 43.2..."


On essaye de lui attribuer ce point

In [51]:
def random_point_in_geometry(poly):

    if isinstance(poly, Polygon) or isinstance(poly,shapely.geometry.MultiPolygon):

        minx, miny, maxx, maxy = poly.bounds
        while True:
            p = Point(random.uniform(minx, maxx), random.uniform(miny, maxy))
            if poly.contains(p):
                return p.x, p.y
            
    else:

        return None
    


# load polygon
geom = df_loc[df_loc.codeCommune == codeInseeCommune_ex].geometryCommune.values[0]

# get random point
x,y = random_point_in_geometry(geom)


# create a map
m = folium.Map(location = [y,x])

# add to map and display 
folium.GeoJson(geom).add_to(m)
folium.Marker(location= [y,x]).add_to(m)

m

On a eu le problème suivant : la commune est encodée par une instance MultiPolygon et non pas (Polygon).

On va donc modifier le code initial en étant moins restrictif

In [75]:
temp = df[(df.y_latitude == 0) & (df.x_longitude == 0)]

temp = pd.merge(temp,df_loc[['codeCommune','geometryCommune']],how='left',right_on='codeCommune',left_on ='codeInseeCommune')

temp["random_point"] = temp["random_point"] = temp.geometryCommune.apply(random_point_in_geometry)

# création de deux colonnes avec les coordonnées x et y
temp[['x_longitude','y_latitude']] = pd.DataFrame(temp.random_point.tolist(), index=temp.index)



On peut rajouter ces nouvelles valeurs à la df initiale

In [88]:
temp.reset_index().merge(df_loc[['codeCommune','geometryCommune']],how='left',right_on='codeCommune',left_on ='codeInseeCommune').set_index('index')

,siret,qualite_xy,y_latitude,x_longitude,nomCommercial,adresse,codeInseeCommune,codeApe,diffusionCommerciale,confiance,codeCommune,geometryCommune
index,,,,,,,,,,,,
17728937,98298254800017,0.0,0.0,0.0,,MARSEILLE 9,13209,68.20B,True,0,13209,"MULTIPOLYGON (((5.45224 43.25746, 5.44966 43.2..."
17728917,98298216700016,0.0,0.0,0.0,,SAINT-CYPRIEN,66171,68.20A,True,0,66171,"MULTIPOLYGON (((3.04356 42.59982, 3.04305 42.6..."
17728906,98298203500015,0.0,0.0,0.0,VICTORIYA,140 RUE DES ANTHEMIS 83600 FREJUS,83061,81.21Z,True,0,83061,"MULTIPOLYGON (((6.87852 43.53272, 6.87599 43.5..."
17728897,98298181300016,0.0,0.0,0.0,,MACON,71270,85.52Z,True,0,71270,"MULTIPOLYGON (((4.84110 46.31199, 4.84467 46.3..."
17728885,98298155700019,0.0,0.0,0.0,,LA CIOTAT,13028,68.20A,True,0,13028,"MULTIPOLYGON (((5.67629 43.19091, 5.67692 43.1..."
...,...,...,...,...,...,...,...,...,...,...,...,...
6685783,389511411,0.0,0.0,0.0,APPAM,73 AV DE STALINGRAD 91120 PALAISEAU,91477,502Z,True,0,91477,"POLYGON ((2.19329 48.71850, 2.19174 48.71232, ..."
7115058,378606834,0.0,0.0,0.0,LES ULIS SERVICES,121 AV DU GENERAL DE GAULLE 92140 CLAMART,92023,502Z,True,0,92023,"POLYGON ((2.22655 48.77610, 2.22707 48.77613, ..."
4774055,352167175,0.0,0.0,0.0,CRECQ,4 RTE DE VIELLA 64350 ARROSES,64056,0121Z,True,0,64056,"POLYGON ((-0.08965 43.55997, -0.08982 43.56030..."


In [71]:
temp = temp.drop(columns=['codeCommune','random_point','geometryCommune'])

# replace elements in df with elements in temp
df.loc[temp.index] = temp


KeyError: '[7407, 7416, 7417, 7423, 7424, 7429, 7431, 7443, 7460, 7461, 7462, 7463, 7464, 7465, 7466, 7467, 7468, 7469, 7470, 7471, 7472, 7473, 7474, 7475, 7476, 7477, 7478, 7479, 7480, 7481, 7482, 7483, 7484, 7485, 7486, 7487, 7488, 7489, 7490, 7491, 7492, 7493, 7494, 7495, 7496, 7497, 7498, 7499, 7500, 7501, 7502, 7503, 7504, 7505, 7506, 7507, 7508, 7509, 7510, 7511, 7512, 7513, 7514, 7515, 7516, 7517, 7518, 7519, 7520, 7521, 7522, 7523, 7524, 7525, 7526, 7527, 7528, 7529, 7530, 7531, 7532, 7533, 7534, 7535, 7536, 7537, 7538, 7539, 7540, 7541, 7542, 7543, 7544, 7545, 7546, 7547, 7548, 7549, 7550, 7551, 7552, 7553, 7554, 7555, 7556, 7557, 7558, 7559, 7560, 7561, 7562, 7563, 7564, 7565, 7566, 7567, 7568, 7569, 7570, 7571, 7572, 7573, 7574, 7575, 7576, 7577, 7578, 7579, 7580, 7581, 7582, 7583, 7584, 7585, 7586, 7587, 7588, 7589, 7590, 7591, 7592, 7593, 7594, 7595, 7596, 7597, 7598, 7599, 7600, 7601, 7602, 7603, 7604, 7605, 7606, 7607, 7608, 7609, 7610, 7611, 7612, 7613, 7614, 7615, 7616

Est ce que ca a bien fonctionné ?

Est ce qu'il reste des établissements non localisés ?